In [4]:
import pandas as pd
from wikidata.client import Client
import wikipedia

import requests
import pickle
from tqdm import tqdm
from requests import utils
from bs4 import BeautifulSoup

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('hungarian')

client = Client()

[nltk_data] Downloading package stopwords to /home/soma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
triplets = pd.read_json('data/poet_triplets.json')
triplets.head()

,source,edge,destination
0,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P31', 'name': 'osztály, amely...","{'wikidata_id': 'Q5', 'name': 'ember', 'descri..."
1,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P21', 'name': 'nem', 'descrip...","{'wikidata_id': 'Q6581097', 'name': 'férfi', '..."
2,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P735', 'name': 'utónév', 'des...","{'wikidata_id': 'Q17498051', 'name': 'József',..."
3,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P570', 'name': 'halálozási id...","{'time': '+1988-02-02T00:00:00Z', 'timezone': ..."
4,"{'name': 'Bencze József', 'wiki_url': '/wiki/B...","{'wikidata_id': 'P106', 'name': 'foglalkozás',...","{'wikidata_id': 'Q49757', 'name': 'költő', 'de..."


In [9]:
triplets.source.apply(lambda d: d['wiki_url']).unique()

array(['/wiki/Bencze_J%C3%B3zsef_(k%C3%B6lt%C5%91)',
       '/wiki/Erg_%C3%81goston', '/wiki/Szokolay_Zolt%C3%A1n', ...,
       '/wiki/Bal%C3%A1zs_F._Attila', '/wiki/Babik_J%C3%B3zsef',
       '/wiki/Grecs%C3%B3_Kriszti%C3%A1n'], dtype=object)

In [10]:
res = requests.get('https://hu.wikipedia.org/wiki/Bencze_J%C3%B3zsef_(k%C3%B6lt%C5%91)')

In [11]:
soup = BeautifulSoup(res.text)

In [12]:
def get_wikipedia_url_from_wikidata_id(wikidata_id, lang='en', debug=False):
    

    url = (
        'https://www.wikidata.org/w/api.php'
        '?action=wbgetentities'
        '&props=sitelinks/urls'
        f'&ids={wikidata_id}'
        '&format=json')
    json_response = requests.get(url).json()
    if debug: print(wikidata_id, url, json_response) 

    entities = json_response.get('entities')    
    if entities:
        entity = entities.get(wikidata_id)
        if entity:
            sitelinks = entity.get('sitelinks')
            if sitelinks:
                if lang:
                    # filter only the specified language
                    sitelink = sitelinks.get(f'{lang}wiki')
                    if sitelink:
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            return requests.utils.unquote(wiki_url)
                else:
                    # return all of the urls
                    wiki_urls = {}
                    for key, sitelink in sitelinks.items():
                        wiki_url = sitelink.get('url')
                        if wiki_url:
                            wiki_urls[key] = requests.utils.unquote(wiki_url)
                    return wiki_urls
    return None   

In [13]:
url = get_wikipedia_url_from_wikidata_id('Q466479', lang='hu')
url

'https://hu.wikipedia.org/wiki/A_helység_kalapácsa'

# TODO
- get poet page contents
- extract sentences with destination entity match (use aliases), mask destination entity in sentence
- create sentence <--> relation database
- train relation classifier

In [22]:
urls = triplets.source.apply(lambda d: d['wiki_url']).unique().tolist()
urls

['/wiki/Bencze_J%C3%B3zsef_(k%C3%B6lt%C5%91)',
 '/wiki/Erg_%C3%81goston',
 '/wiki/Szokolay_Zolt%C3%A1n',
 '/wiki/Payer_Imre_(k%C3%B6lt%C5%91)',
 '/wiki/N%C3%B3gr%C3%A1di_G%C3%A1bor',
 '/wiki/Fekete_Anna',
 '/wiki/Bl%C3%A1zy_J%C3%A1nos_(lelk%C3%A9sz,_1684%E2%80%931763)',
 '/wiki/Orb%C3%A1n_Ott%C3%B3',
 '/wiki/Pardi_Anna',
 '/wiki/Zakari%C3%A1s_S%C3%A1ndor',
 '/wiki/Feny%C5%91_L%C3%A1szl%C3%B3_(k%C3%B6lt%C5%91)',
 '/wiki/Bal%C3%A1zs_Andr%C3%A1s_(k%C3%B6lt%C5%91)',
 '/wiki/Kacs%C3%B3_S%C3%A1ndor_(%C3%BAjs%C3%A1g%C3%ADr%C3%B3)',
 '/wiki/D%C3%A9k%C3%A1ny_D%C3%A1vid',
 '/wiki/Toldalagi_P%C3%A1l',
 '/wiki/J%C3%A1n_Andrej_Demian',
 '/wiki/%C3%9Ajv%C3%A1ry_S%C3%A1ndor',
 '/wiki/Monoszl%C3%B3y_Dezs%C5%91',
 '/wiki/Tornai_J%C3%B3zsef',
 '/wiki/Bajor_Andor',
 '/wiki/Erd%C3%A9lyi_Zolt%C3%A1n_(%C3%BAjs%C3%A1g%C3%ADr%C3%B3)',
 '/wiki/Szab%C3%B3_Gyula_(fest%C5%91)',
 '/wiki/N%C3%A9methi_Ferenc',
 '/wiki/Zsille_G%C3%A1bor',
 '/wiki/Fekete_J%C3%A1nos_(kamar%C3%A1s)',
 '/wiki/Cs%C3%A1k_Gy%C3%B6ngyi',
 '/

In [23]:
def get_wiki_lines(url_suffix):
    url = 'https://hu.wikipedia.org' + url_suffix
    
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    
    cont = soup.find('div', {'class': 'mw-parser-output'})
    lines = [l for l in cont.text.split('\n') if l]
    
    return lines

In [30]:
lines_mapping = {}

for url in tqdm(urls):
    try:
        lines = get_wiki_lines(url)
        lines_mapping[url] = lines
    except:
        print('Error, passing', url)

100%|██████████| 2052/2052 [25:02<00:00,  1.37it/s]


In [1]:
import pickle

In [32]:
with open('data/poet_articles.p', 'wb') as f:
    pickle.dump(lines_mapping, f)

In [5]:
with open('data/poet_articles.p', 'rb') as f:
    lines_mapping = pickle.load(f)

In [6]:
source_url = '/wiki/Pilinszky_J%C3%A1nos'

In [7]:
def get_wikitext_matches_by_ratio(source_url):

    destinations = triplets[triplets.source.apply(lambda d: d['wiki_url'] == source_url)][['edge', 'destination']].values

    res = []

    split_l = [ll.split('.') for ll in lines_mapping[source_url]]
    flat_split_l = [item for sublist in split_l for item in sublist]

    for e, d in destinations:
        if 'description' in e:
            rel_words = set([w for w in e['description'].split(' ') + e['aliases'] if w not in stopwords])

            if 'name' in d:
                tax = [d['name']] + d['aliases']
            elif 'time' in d:
                tax = [d['time']]
            else:
                continue

            for l in flat_split_l:
                if len(l.split()) < 5:
                    continue
                l_words = set([w for w in l.split(' ') if w not in stopwords])
                for t in tax:
                    if t.lower() in l.lower():
                        res.append((e['wikidata_id'], t, l, len(l_words.intersection(rel_words))/len(l_words)))
                        break

    df = pd.DataFrame(res, columns=['relation', 'match', 'sent', 'match_ratio']).reset_index()
    df_max_ratio = df[df.match_ratio > 0.1].groupby(by='relation').match_ratio.max().rename('max_match_ration')
    df_w_max = df.merge(df_max_ratio, on=['relation'])
    df_w_max = df_w_max[df_w_max.match_ratio == df_w_max.max_match_ration]

    return df_w_max

In [3]:
def get_wikitext_matches(source_url):
    destinations = triplets[triplets.source.apply(lambda d: d['wiki_url'] == source_url)][['edge', 'destination']].values

    res = []

    split_l = [ll.split('.') for ll in lines_mapping[source_url]]
    flat_split_l = [item for sublist in split_l for item in sublist]

    for l in flat_split_l:
        if len(l.split()) < 5:
            continue

        sub_res = []
        for e, d in destinations:
            if 'name' in d:
                tax = [d['name']] + d['aliases']
            elif 'time' in d:
                tax = [d['time']]
            else:
                continue

            for t in tax:
                if t.lower() in l.lower():
                    sub_res.append((e['wikidata_id'], t, l))
                    break

        if len(sub_res) == 1:
            res.append(sub_res[0])

    res_df = pd.DataFrame(res, columns=['relation', 'match', 'sent']).groupby(by='match').first().reset_index()
    return res_df

In [8]:
res = []

for url in lines_mapping:
    res.append(get_wikitext_matches_by_ratio(url))

In [9]:
res_df = pd.concat(res)
res_df

,index,relation,match,sent,match_ratio,max_match_ration
4,49,P1412,hu,"de ennek is már húsz-huszonkét éve,",0.333333,0.333333
0,26,P19,Budapest,"Orbán Ottó született Szauer Ottó (Budapest, 1936",0.166667,0.166667
0,5,P19,Budapest,"Újváry Sándor, született Ungár Sándor (Budapes...",0.125000,0.125000
0,36,P119,Házsongárdi temető,"Bajor Andor sírja Kolozsvárott, a Házsongárdi...",0.142857,0.142857
14,26,P6275,,Mestere volt Bacskai Bélának és Dúdor Istvánnak,0.166667,0.166667
...,...,...,...,...,...,...
0,2,P735,József,Erődi Dániel JózsefÉletrajzi adatokSzületési név,0.200000,0.200000
2,25,P463,Magyar PEN Club,A Magyar PEN Club tagja,0.200000,0.200000
125,148,P463,,A Magyar PEN Club tagja,0.200000,0.200000
130,153,P463,,A Romániai Írók Szövetségének tagja,0.200000,0.200000


In [10]:
sents = res_df.groupby(by='relation').sent.apply(list)

In [11]:
sents_dct = sents.to_dict()

In [19]:
sents_dct['P1343']

[' Számos ritka Biblia-kiadás is található a gyűjteményben',
 ' Számos ritka Biblia-kiadás is található a gyűjteményben',
 ' Jelentős vers minden költői szakaszában található',
 ' Jelentős vers minden költői szakaszában található',
 ' Jelentős vers minden költői szakaszában található',
 ' Jelentős vers minden költői szakaszában található',
 ' Jelentős vers minden költői szakaszában található',
 ' Jelentős vers minden költői szakaszában található',
 'Flóra édesanyja a Városligeti tavon található Páva-szigetet[2] bérelte ki Ifjú korában',
 'József Attila-szobor, Ózd: a József Attila Gimnázium előterében található',
 ' Radnóti ebben a indította el Eklogák sorozatát, itt található az Első ecloga',
 ' Sírja jelenleg a németországi Bermattingen falu temetőjében található',
 'Új magyar irodalmi lexikon, Péter László szerk',
 ' Sírja a Kerepesi úti temetőben található',
 '↑ Magyar életrajzi lexikon, Pilinszky Zsigmond',
 '↑ Magyar életrajzi lexikon, Pilinszky Géza',
 ' A Mészáros utcai ház fal

In [12]:
sents.apply(len)

relation
P101       1
P102       8
P103      44
P1038      5
P106       5
P119      80
P1343     48
P1344      1
P1412     39
P1598      3
P166       1
P172       1
P19      122
P20        2
P21       35
P26        2
P264       1
P31        1
P3448      1
P39        1
P40        9
P463      51
P485       2
P509      12
P6275     40
P6379     57
P641       1
P6886     42
P69        2
P7084      2
P735      22
P910      42
P97        3
Name: sent, dtype: int64